# 40 – Build Draft Report

In [ ]:
from datetime import datetime
dbutils.widgets.text("CATALOG","<CATALOG_NAME>")
catalog = dbutils.widgets.get("CATALOG")
spark.sql(f"USE CATALOG {catalog}")
spark.sql("USE SCHEMA gold")

run_id = f"RR_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"
spark.sql(f"INSERT INTO gold.report_runs VALUES ('{run_id}','Risk Report','DRAFT','rules@current',current_timestamp(),NULL,NULL,NULL)")

spark.sql("""
CREATE OR REPLACE TEMP VIEW _kpi AS
SELECT 'loans_total' as metric, 'all' as dimension, count(*)*1.0 as value FROM gold.risk_eval
UNION ALL
SELECT 'high_risk_count','all', sum(CASE WHEN risk_band='High' OR risk_points>=20 THEN 1 ELSE 0 END) FROM gold.risk_eval
UNION ALL
SELECT 'avg_dti','all', avg(dti) FROM gold.risk_eval
UNION ALL
SELECT 'avg_fico','all', avg(fico_score) FROM gold.risk_eval
""")

spark.sql(f"DELETE FROM gold.report_facts WHERE report_run_id = '{run_id}'")
spark.sql(f"INSERT INTO gold.report_facts SELECT '{run_id}', metric, dimension, value FROM _kpi")
print(f"Draft report created: {run_id}")
